In [1]:
import gspread
import pandas as pd
import requests
import time
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
from gspread_dataframe import  get_as_dataframe, set_with_dataframe
from google.auth import default


# Authenticate and create the PyDrive client and authenticate the spreadsheet editing 
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)
creds, _ = default()
gc = gspread.authorize(creds)

In [2]:
# connecting to the google sheet
puig_images = gc.open_by_url('https://docs.google.com/spreadsheets/d/1A9CYWcmHWecRxm6JWx-2dHVls1NOEcMGZFSFeqkCXyA/edit#gid=1241926433')
#spesifying the worksheet
ws = puig_images.worksheet('Images Missed')
#pulling the data into a pandas dataframe
df = get_as_dataframe(ws,'r')
#Removing all NAN values i.e. empty records
df = df.dropna(how='all')

In [3]:
def get_names(soup):
  names_list = []
  for tag in soup.find_all('a'):
    image_temp = str(tag.get('download'))
    #print(image_temp)
    if image_temp != "None":
      image_temp = image_temp.split('_')
      names_list.append(image_temp[0])
  return names_list
#If the number of parents is >=10 note the sku for possible errors
def error_watch(sku,url):
  %cd ..
  f = open("error-watch.docx", "a")
  record = "SKU: " + str(sku) + " URL: " + str(url) + "\n"
  f.write(record)
  f.close()

In [4]:
#Expt cell
import requests
from bs4 import BeautifulSoup
import os
import os.path
from IPython. display import clear_output
import re
import random
!pip install emoji
import emoji
#random HTTP header
headers = {
    'Accept-Encoding': 'gzip, deflate, sdch',
    'Accept-Language': 'en-US,en;q=0.8',
    'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_10_1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/39.0.2171.95 Safari/537.36',
    'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,*/*;q=0.8',
    'Referer': 'http://www.wikipedia.org/',
    'Connection': 'keep-alive',
}
#Creating and navigating to puig directory
!mkdir puig
%cd puig/
e_list = ['\U0001F634\U0001F634\U0001F634','\U0001F635\U0001F635','\U0001F40C\U0001F40C\U0001F40C','\U0001F9A5\U0001F9A5\U0001F9A5','\U0001F62A\U0001F62A\U0001F62A\U0001F62A','\U0001F62D\U0001F62D\U0001F62D\U0001F62D','\U0001F62B\U0001F62B']
progress_count = 1
#Iterating through all SKU
for index in range(len(df)):
  #Use the next line to test individual SKU
  #if str(df.loc[index, 'sku']) == "5627O":
    #temp_list stores all the "PRODUCT image" paths to avoid duplication as the page may have multiple instances of the same image
    temp_list = []
    clear_output()
    print(str(progress_count) +' of '+ str(len(df)) +' done.' + random.choice(e_list))
    progress_count = progress_count + 1
    #Iterator for names_list 
    i = 0
    #image number counter for repeated SKU
    j = 1
    #repeated SKU counter:02 onwards for parent and onwards 
    repeat = 2
    #Total image counter per sku
    image_count = 0
    #Extracting the URL to the SKU
    sku = df.loc[index, 'sku']
    #Extracting the URL to the SKU
    url = df.loc[index, 'Image URL']
    #Get request to the puig server
    r = requests.get(url=url, headers=headers)
    soup = BeautifulSoup(r.text, 'html.parser')
    #nemes/SKU/Color of the products to be used for naming the images
    names_list = get_names(soup)
    #print(names_list)
    #PUIG stores the "PRODUCT images" in a folder named products
    substring = 'products'
    path = "/content/puig/"+str(df.loc[index, 'sku'])
    #creating and navigating to the directories for each SKU
    os.mkdir(path)
    os.chdir(path)
    #Iterating through all the images
    for img in soup.find_all('img'):
      #extracting image path
      image_all = img.get('src')
      #checking if it is a "PRODUCT image"
      if substring in image_all:
        image_temp = img.get('src')
        #images tags/paths maybe repeated on the website for creating thumbnails or preview images 
           
        if image_temp not in temp_list:
          #paths stariing with "/" are incomplete
          if image_temp[:1] == '/':
            #completing the path 
            image_path = 'https://puig.tv/' + image_temp
            #naming the images as per nomenclature
            #1st image downloaded from puig is parent 
            try:
              if i == 0:
                #saving the 1st image as parent 
                string = names_list[i]
                #removing non-numeric letters from SKU 
                parent_name = re.sub(r'[^0-9]', '', string.lower())
                name = 'PG' + parent_name + '-0' + str(repeat)
                repeat = repeat + 1
                with open('{}.jpg'.format(name), 'wb') as f:
              #writing/downloading the image to collab drive instance
                 f.write(requests.get(url=image_path).content)
                #saving the 1st image again for color 
                name = 'PG' + names_list[i]
              else:
                name = 'PG' + names_list[i]
              if os.path.exists(name+".jpg"):
                string = names_list[i]
                parent_name = re.sub(r'[^0-9]', '', string.lower())
                name = 'PG' + parent_name + '-0' + str(repeat)
                repeat = repeat + 1
              i = i + 1  
            except:
              #managing exception when there's no sku name refrence on website
              string = str(df.loc[index, 'sku'])
              parent_name = re.sub(r'[^0-9]', '', string.lower())
              name = 'PG' + parent_name + '-0' + str(repeat)
              repeat = repeat + 1
              #writing/downloading the image to collab drive instance
              with open('{}.jpg'.format(name), 'wb') as f:
                 f.write(requests.get(url=image_path).content)
              name = 'PG'+str(df.loc[index, 'sku'])
              repeat = repeat + 1
            with open('{}.jpg'.format(name), 'wb') as f:
              #writing/downloading the parent image to collab drive instance
              f.write(requests.get(url=image_path).content)
            temp_list.append(image_temp)
            image_count = image_count + 1
    #Some pages show multiple sku or accesories to sku  or poorly coded html hence need manual matching
    if (image_count) > len(names_list) and len(names_list) > 0:
      error_watch(sku,url)
print("Download Complete.")

129 of 1673 done.😪😪😪😪


KeyboardInterrupt: ignored

In [ ]:
#creating a zip file for download
!zip -r /content/puig.zip /content/puig

In [ ]:
#backup

"""import requests
from bs4 import BeautifulSoup
import os
from IPython. display import clear_output
#random HTTP header
headers = {
    'Accept-Encoding': 'gzip, deflate, sdch',
    'Accept-Language': 'en-US,en;q=0.8',
    'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_10_1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/39.0.2171.95 Safari/537.36',
    'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,*/*;q=0.8',
    'Referer': 'http://www.wikipedia.org/',
    'Connection': 'keep-alive',
}
#Creating and navigating to puig directory
!mkdir puig
%cd puig/

progress_count = 1
#Iterating through all SKU
for index in range(len(df)):
  temp_list = []
  clear_output()
  print(str(progress_count) +' of '+ str(len(df)) +' completed.')
  progress_count = progress_count + 1
  i = 1
  #Extracting the URL to the SKU
  url = df.loc[index, 'Image URL']
  #Get request ot the puig server
  r = requests.get(url=url, headers=headers)
  soup = BeautifulSoup(r.text, 'html.parser')
  #PUIG stores the "PRODUCT images" in a folder named default or names the images as default
  substring = 'default'
  path = "/content/puig/"+str(df.loc[index, 'sku'])
  #creating and navigating to the directories for each SKU
  os.mkdir(path)
  os.chdir(path)
  #Iterating through all the images
  for img in soup.find_all('img'):
    #extracting image path
    image_all = img.get('src')
    #checking if it is a "PRODUCT image"
    if substring in image_all:
      image_temp = img.get('src')
      #images tags/paths maybe repeated on the website for creating thumbnails or preview images 
      #temp_list stores all the "PRODUCT image" paths to avoid duplication   
      if image_temp not in temp_list:
        #paths stariing with "/" are incomplete
        if image_temp[:1] == '/':
          #completing the path 
          image_path = 'https://puig.tv/' + image_temp
          #naming the images as per nomenclature
          name = 'PG'+str(df.loc[index, 'sku'])+'-0' + str(i)
          i = i + 1
          with open('{}.jpg'.format(name), 'wb') as f:
            #writing/downloading the image to collab drive instance
            f.write(requests.get(url=image_path).content)
          temp_list.append(image_temp)
  %cd .."""
    